# Build an advance ChatBot with session memory using LangChain.

#### Specifications:

* Will be able to have a conversation.
* Will remember the previous interactions, will have memory.
* Will be able to have different memories for different user sessions.
* Will be able to remember limited number of messages, limited memory.

### Avoid Nasty Deprecation Warnings from LangChain.

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

### Connect with .env file for API key access.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

### Connect with LLM model and start a conversation with it.

* For this project, we will use OpenSource llama3-70b-8192 model.

In [3]:
from langchain_groq import ChatGroq

chatbot = ChatGroq(model="llama3-70b-8192")

* HumanMessage: The User Input

In [4]:
from langchain_core.messages import HumanMessage

messageToTheChatbot = [
    HumanMessage(content="My favorite color is blue")
]

In [5]:
chatbot.invoke(messageToTheChatbot)

AIMessage(content="Blue is a wonderful color! It's often associated with feelings of calmness, trust, and serenity. Did you know that blue is also one of the most popular favorite colors among people?\n\nWhat is it about blue that you love the most? Is it a particular shade of blue, like sky blue, navy blue, or royal blue? Or is it the way blue makes you feel?\n\nLet's chat more about your love for blue!", response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 15, 'total_tokens': 105, 'completion_time': 0.257343164, 'prompt_time': 0.00332196, 'queue_time': 0.015847100000000003, 'total_time': 0.260665124}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-31596ac7-2d3e-487f-af04-3ab2a942e40f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 90, 'total_tokens': 105})

### Check if the ChatBot remembers your favorite color.

In [6]:
chatbot.invoke([HumanMessage(content="What is my favorite color?")])

AIMessage(content="I'm just an AI, I don't have have access to personal information about you, so I don't know what your favorite color is. Would you like to tell me?", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 16, 'total_tokens': 53, 'completion_time': 0.106303738, 'prompt_time': 0.003422986, 'queue_time': 0.016881442, 'total_time': 0.109726724}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_2f30b0b571', 'finish_reason': 'stop', 'logprobs': None}, id='run-c63ee979-dcbd-4b68-b087-c5ea55a310ad-0', usage_metadata={'input_tokens': 16, 'output_tokens': 37, 'total_tokens': 53})

### Let's add memory to our ChatBot

* We will use the ChatMessageHistory package.
* We will save the ChatBot memory in a python dictionary call chatbotMemory
* We will define the get_session_history function to create a session_id for each conversation.
* We will use the built-in runnable RunnableWithMessageHistory.

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] =ChatMessageHistory()
    return chatbotMemory[session_id]

chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot,
    get_session_history
)

### RunnableWithMessageHistory

##### When invoking a new RunnableWithMessageHistory, we specify the corresponding chat history using a configurable parameter. Let's say we want to create a chat memory for one user session, let's call it session1:

In [8]:
session1 = {"configurable": {"session_id":"001"}}

In [9]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is orange")],
    config = session1,
)
responseFromChatbot.content

"Orange is a vibrant and energetic color! It's often associated with feelings of warmth, excitement, and creativity. Do you have a particular shade of orange that you're especially fond of, or is it more of a general love for the color?\n\nAlso, what is it about orange that resonates with you? Is it a color that reminds you of a specific memory or experience, or do you just love the way it looks?"

In [10]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)
responseFromChatbot.content

'I remember! Your favorite color is orange!'

### Now, Let's change the session_id and see what happen.

* Now let's create a chat memory for another user session, let's call it session2:

In [11]:
session2 = {"configurable": {"session_id":"002"}}

* If the chatbot uses this new memory for session2, it will not be able to remember anything from the previous conversation in the session1:

In [12]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session2,
)
responseFromChatbot.content

"I'm just an AI, I don't have the ability to know personal information about you, including your favorite color. However, I can ask you to tell me what your favorite color is if you'd like to share!"

### Let's go back to session1 and see if memory is still there or not.

In [13]:
session1 = {"configurable": {"session_id":"001"}}

In [14]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)
responseFromChatbot.content

'You told me earlier that your favorite color is orange!'

* As we can see now, the chatbot is able to remember session1 conversation.